In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
import sys
import os
import pickle

# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Load environment variables from .env file
load_dotenv()
from src.config_loader import config
from src.utils import create_html_story, create_test_story_dict
from src.audio_generation import text_to_speech


In [20]:
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_nonsilent
import os
import tempfile
from google.cloud import texttospeech
import re

class SSMLWordClipper:
    def __init__(self, break_strength='strong', break_time='250ms'):
        """
        Initialize the SSML Word Clipper.
        
        Args:
            break_strength (str): SSML break strength ('none', 'x-weak', 'weak', 
                                'medium', 'strong', 'x-strong')
            break_time (str): Break time in milliseconds (e.g., '250ms')
        """
        self.client = texttospeech.TextToSpeechClient()
        self.break_strength = break_strength
        self.break_time = break_time
        
    def create_ssml(self, text):
        """
        Convert plain text to SSML with breaks between words.
        
        Args:
            text (str): Input text
            
        Returns:
            str: SSML formatted text with breaks between words
        """
        # Clean the input text
        text = text.strip()
        
        # Split into words while preserving punctuation
        words = re.findall(r'\b\w+\b|[.,!?;]', text)
        
        # Create SSML with breaks
        ssml_parts = ['<speak>']
        
        for i, word in enumerate(words):
            ssml_parts.append(word)
            
            # Add break after each word except the last one and punctuation
            if i < len(words) - 1 and not re.match(r'[.,!?;]', words[i + 1]):
                ssml_parts.append(
                    f'<break strength="{self.break_strength}" time="{self.break_time}"/>'
                )
                
        ssml_parts.append('</speak>')
        
        return ' '.join(ssml_parts)
    
    def synthesize_speech(self, text, output_file="output.mp3"):
        """
        Convert text to speech with distinct word breaks using SSML.
        
        Args:
            text (str): Input text
            output_file (str): Path to save the output audio file
            
        Returns:
            str: Path to the output audio file
        """
        # Generate SSML
        ssml_text = self.create_ssml(text)
        
        # Configure the voice
        voice = texttospeech.VoiceSelectionParams(
            language_code="en-US",
            name="en-US-Neural2-D",  # Using a neural voice for better quality
            ssml_gender=texttospeech.SsmlVoiceGender.MALE
        )
        
        # Configure the audio
        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3,
            speaking_rate=0.7  # Normal speaking rate
        )
        
        # Create synthesis input
        synthesis_input = texttospeech.SynthesisInput(ssml=ssml_text)
        
        # Perform the text-to-speech request
        response = self.client.synthesize_speech(
            input=synthesis_input,
            voice=voice,
            audio_config=audio_config
        )
        
        self.audio = response.audio_content
        # Write the response to the output file
        with open(output_file, "wb") as out:
            out.write(response.audio_content)
            
        return output_file



In [21]:
    # Basic usage
# Default configuration (strong breaks, 250ms)
clipper = SSMLWordClipper()

# Example 2: Longer breaks
clipper_long = SSMLWordClipper(break_strength='x-strong', break_time='100ms')
clipper_long.synthesize_speech(text, "output_long_breaks.mp3")



'output_long_breaks.mp3'

In [ ]:
clipper_long.audio

b'\xff\xf3\x84\xc4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xf3\x84\xc4\x00%Z\x96\x10\x00a\x86\xb9\x01\x8e\x1c\x18\x08\x00qD&`\xd0D&9\xd8\xb2\xa7\xe8 ,/\x9e\x9f\xb0@\x88\x82\x11\x04\x08\x13\xb2ad\xec\x98Zq\x11\x11\x06F=\xdf\xefw\xb7\xb9\xcc\x00\x10\x81\x00\xdd\xdf\xae\xe1\xc4p\x00B"&\

In [51]:
story_data = create_test_story_dict(story_data, 1, 1)

In [9]:
from src.anki_tools import generate_wiktionary_links_non_english


def test_wiktionary_links():
    """Test function to demonstrate usage."""
    test_cases = [
        ("goodbye England", "uk"),  # Ukrainian
        ("book reading", "sv"),  # Swedish
        ("coffee shop", "ja"),  # Japanese
    ]
    
    for phrase, lang_code in test_cases:
        print(f"\nTesting {lang_code} Wiktionary links for: {phrase}")
        result = generate_wiktionary_links_non_english(phrase, lang_code)
        print(f"Result: {result}")

In [13]:
test_wiktionary_links()


Testing uk Wiktionary links for: goodbye England
native url is: https://uk.wiktionary.org/wiki/goodbye
native url is: https://uk.wiktionary.org/wiki/england
Result: <a href="https://uk.wiktionary.org/wiki/goodbye#Англійська">goodbye</a> <a href="https://en.wiktionary.org/wiki/england#English">England</a>

Testing sv Wiktionary links for: book reading
native url is: https://sv.wiktionary.org/wiki/book
native url is: https://sv.wiktionary.org/wiki/reading
Result: <a href="https://sv.wiktionary.org/wiki/book#Engelska">book</a> <a href="https://sv.wiktionary.org/wiki/reading#Engelska">reading</a>

Testing ja Wiktionary links for: coffee shop
native url is: https://ja.wiktionary.org/wiki/coffee
native url is: https://ja.wiktionary.org/wiki/shop
Result: <a href="https://ja.wiktionary.org/wiki/coffee#英語">coffee</a> <a href="https://ja.wiktionary.org/wiki/shop#英語">shop</a>
